#### Import libraries

In [140]:
import librosa
import numpy as np
import os
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import time
from scipy.fftpack import fft, ifft
from scipy.signal import find_peaks

#### Normalized Energy

In [2]:
def compute_normalized_energy(frames):
    """Tính năng lượng chuẩn hóa của từng khung."""
    energy = np.sum(frames**2, axis=1)
    if np.max(energy) > 0:
        normalized_energy = energy / np.max(energy)  # Chuẩn hóa nếu max > 0
    else:
        normalized_energy = energy  # Giữ nguyên nếu max = 0

    return normalized_energy

#### Caculated F0 Using HPS

In [141]:
def find_f0(frame, fs, N_FFT=1024):
    # Áp dụng cửa sổ Hamming
    windowed_frame = frame * np.hamming(len(frame))
    
    # 2. Chuyển đổi Fourier nhanh (FFT)
    spectrum = fft(windowed_frame, N_FFT)
    
    # 3. Tính log phổ (Log Spectrum)
    log_spectrum = np.log(np.abs(spectrum) + np.finfo(float).eps)  # Thêm epsilon để tránh log(0)
    
    # 4. Biến đổi ngược Fourier (Inverse FFT) để thu được Cepstrum
    cepstrum = np.real(ifft(log_spectrum))
    
    # 5. Tìm đỉnh cepstrum trong khoảng quefrency tương ứng với F0
    quefrency_range = int((1 / 700) * fs), int((1 / 30) * fs)  # Khoảng 60 Hz đến 400 Hz
    weighted_cepstrum = cepstrum[quefrency_range[0]:quefrency_range[1]] * np.linspace(1, 1, quefrency_range[1] - quefrency_range[0])

    # 6. Tìm đỉnh trong khoảng quefrency liên quan đến F0
    cepstrum_peak_index = np.argmax(weighted_cepstrum) + quefrency_range[0]
    peaks, _ = find_peaks(cepstrum[quefrency_range[0]:quefrency_range[1]], height=0.1, distance=10)
    if len(peaks) > 0:
        cepstrum_peak_index = peaks[-1] + quefrency_range[0]
    else:
        cepstrum_peak_index = 0
    f0 = fs / cepstrum_peak_index if cepstrum_peak_index != 0 else 0    
    return f0

In [176]:
def difference_function_fast(signal, max_lag):
    """Tính hàm hiệu sai nhanh bằng NumPy."""
    size = len(signal)
    padded_signal = np.pad(signal, (0, max_lag), mode='constant', constant_values=0)
    cumsum = np.cumsum(padded_signal**2)
    diff = np.zeros(max_lag)
    for tau in range(1, max_lag):
        # Hiệu quả hơn bằng cách sử dụng cumsum và dot product
        diff[tau] = cumsum[size] - cumsum[tau] - 2 * np.dot(signal[:size-tau], signal[tau:size])
    return diff

def cumulative_mean_normalized_difference_fast(diff):
    """Chuẩn hóa hàm hiệu sai tích lũy (phiên bản nhanh)."""
    cmndf = np.zeros_like(diff)
    cmndf[0] = 1  # Không sử dụng tau = 0
    running_sum = 0
    for tau in range(1, len(diff)):
        running_sum += diff[tau]
        cmndf[tau] = diff[tau] / (running_sum / tau) if running_sum > 0 else 1
    return cmndf

def find_f0_yin(signal, sr, min_freq=50, max_freq=700, threshold=0.1):
    """Tính F0 bằng phương pháp YIN (tối ưu)."""
    # Tính độ trễ tối đa và tối thiểu
    max_lag = int(sr / min_freq)
    min_lag = int(sr / max_freq)

    # Tính hàm hiệu sai nhanh
    diff = difference_function_fast(signal, max_lag)

    # Chuẩn hóa hàm hiệu sai
    cmndf = cumulative_mean_normalized_difference_fast(diff)

    # Tìm độ trễ (tau) đầu tiên vượt qua ngưỡng
    tau = absolute_threshold(cmndf[min_lag:], threshold)
    tau += min_lag  # Bù chỉ số vì cmndf bị cắt

    # Nếu không tìm được độ trễ hợp lệ
    if tau == 0:
        return 0

    # Tính F0 từ độ trễ
    f0 = sr / tau

    # Nếu F0 nằm ngoài khoảng cho phép, trả về 0
    if f0 < min_freq or f0 > max_freq:
        return 0

    return f0

# Hàm tìm ngưỡng không đổi (giữ nguyên)
def absolute_threshold(cmndf, threshold):
    for tau in range(1, len(cmndf)):
        if cmndf[tau] < threshold:
            return tau
    return 0

#### Detect Baby Cry

In [169]:
def detect_audio_class(file_path, frame_length_ms=30, frame_step_ms=15, sr=16000, 
                       energy_threshold=0.005, f0_threshold=400):

    signal, sr = librosa.load(file_path, sr=sr)

    frame_length = int(frame_length_ms * sr / 1000)
    frame_step = int(frame_step_ms * sr / 1000)
    frames = librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_step).T
    
    normalized_energy = compute_normalized_energy(frames)

    silence_count = 0
    voice_count = 0
    cry_count = 0

    for frame, energy in zip(frames, normalized_energy):
        if energy < energy_threshold:
            silence_count += 1
        else:
            f0 = find_f0_yin(frame, sr)
            if f0 > f0_threshold:
                cry_count += 1
            elif f0 == 0:
                continue
            else:
                voice_count += 1

    print(f"Frames classified as Silence: {silence_count}")
    print(f"Frames classified as Voice: {voice_count}")
    print(f"Frames classified as Cry: {cry_count}")

    if silence_count > max(voice_count, cry_count):
        if voice_count > cry_count:
            return "Adult Voice"
        else:
            return "Baby Cry"
    elif cry_count > voice_count:
        return "Baby Cry"
    else:
        return "Adult Voice"

In [179]:
if __name__ == "__main__":
    file_path = "temp_record.wav"
    
    classification = detect_audio_class(file_path)
    print(f"Audio is classified as: {classification}")

103.8961038961039
101.26582278481013
100.62893081761007
100.62893081761007
102.56410256410257
102.56410256410257
102.56410256410257
96.3855421686747
571.4285714285714
592.5925925925926
0
0
50.314465408805034
109.58904109589041
118.51851851851852
126.98412698412699
0
0
0
0
0
0
102.56410256410257
112.67605633802818
110.34482758620689
108.10810810810811
107.38255033557047
108.84353741496598
109.58904109589041
111.11111111111111
216.21621621621622
108.10810810810811
107.38255033557047
200.0
205.12820512820514
108.10810810810811
202.53164556962025
107.38255033557047
108.84353741496598
108.84353741496598
105.96026490066225
105.96026490066225
103.2258064516129
103.2258064516129
113.47517730496453
99.37888198757764
207.7922077922078
0
0
0
103.8961038961039
101.26582278481013
102.56410256410257
101.91082802547771
100.62893081761007
100.0
98.15950920245399
94.67455621301775
592.5925925925926
533.3333333333334
87.91208791208791
86.48648648648648
0
0
0
66.11570247933884
98.76543209876543
0
0
0
0
0

In [6]:
def evaluate_audio_classification(dataset_path, frame_length_ms=30, frame_step_ms=15, 
                                  sr=16000, energy_threshold=0.02, f0_threshold=300):
    true_labels = []
    predicted_labels = []
    
    class_map = {'adult voice': 'Adult Voice', 'baby cry': 'Baby Cry'}

    # Tổng thời gian thực thi cho tất cả các tệp
    total_time = 0
    total_files = 0
    file_times = []  # Dùng để lưu trữ thời gian xử lý từng tệp
    
    start_time = time.time()  # Bắt đầu tính thời gian toàn bộ quá trình

    for class_name, expected_label in class_map.items():
        class_folder = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_folder):
            print(f"Directory not found: {class_folder}")
            continue
        
        print(f"Processing class: {class_name}")
        
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            
            # Bắt đầu đo thời gian cho từng tệp
            file_start_time = time.time()

            predicted_label = detect_audio_class(
                file_path, 
                frame_length_ms=frame_length_ms, 
                frame_step_ms=frame_step_ms, 
                sr=sr, 
                energy_threshold=energy_threshold, 
                f0_threshold=f0_threshold
            )
            
            # Ghi nhãn thực và nhãn dự đoán
            true_labels.append(expected_label)
            predicted_labels.append(predicted_label)

            # Tính thời gian cho file hiện tại
            file_end_time = time.time()
            file_time = file_end_time - file_start_time
            total_time += file_time
            total_files += 1

            # Lưu thời gian vào danh sách
            file_times.append(file_time)

    # Kết thúc đo thời gian
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Tính độ chính xác
    accuracy = accuracy_score(true_labels, predicted_labels) * 100
    print("Độ chính xác: {:.2f}%".format(accuracy))

    # Tính và in độ chính xác theo từng lớp
    print("\nĐộ chính xác mỗi lớp:")
    for label in class_map.values():
        class_true = [1 if l == label else 0 for l in true_labels]
        class_pred = [1 if l == label else 0 for l in predicted_labels]
        class_accuracy = accuracy_score(class_true, class_pred) * 100
        print(f"- {label}: {class_accuracy:.2f}%")

    # Báo cáo chi tiết
    # print("\nClassification Report:")
    print(classification_report(true_labels, predicted_labels, target_names=class_map.values(), zero_division=0))

    # # In tổng thời gian cho tất cả quá trình đánh giá
    print(f"\nTổng thời gian xử lý: {elapsed_time:.4f} seconds")

    # # Tính thời gian trung bình xử lý mỗi tệp
    if total_files > 0:
        avg_time_per_file = total_time / total_files
        std_dev_time_per_file = np.std(file_times)  # Tính độ lệch chuẩn từ danh sách file_times
        print(f"Trung bình (Mean) của 1 tệp: {avg_time_per_file:.4f} seconds")
        print(f"Độ lệch chuẩn (Std Dev) của 1 tệp: {std_dev_time_per_file:.4f} seconds")
        
    # Ma trận nhầm lẫn
    cm = confusion_matrix(true_labels, predicted_labels, labels=list(class_map.values()))
    print("\nMa trận nhầm lẫn:")
    print(cm)

In [ ]:
if __name__ == '__main__':
    dataset_testing_path = 'datasets_testing'
    evaluate_audio_classification(dataset_testing_path)